In [1]:
import numpy as np
import scipy as sp
import seaborn as sns
import random
from matplotlib import pyplot as plt
from openpyxl import Workbook
write_wb = Workbook()
write_ws = write_wb.active

class UGMM(object):
    '''Univariate GMM with CAVI'''
    def __init__(self, X, K=2, sigma=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]
        self.sigma2 = sigma**2

    def _init(self):
        self.phi = np.random.dirichlet([np.random.random()*np.random.randint(1, 10)]*self.K, self.N)
        self.m = np.random.randint(int(self.X.min()), high=int(self.X.max()), size=self.K).astype(float)
        self.m += self.X.max()*np.random.random(self.K)
        self.s2 = np.ones(self.K) * np.random.random(self.K)
        bias = 1
        std = 2
        x = np.array([7.54,4.8])
        x = np.array([15*random.random(),15*random.random()])
        #random.random()
        H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
        H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
        d = x-H
        d = np.multiply(d,d)
        d = np.sqrt(d.sum(axis=1))
        d[0] = d[0] + np.random.normal(bias,2,1)
        d[1] = d[1] + np.random.normal(bias,2,1)
        d[2] = d[2] + np.random.normal(bias,2,1)
        d[3] = d[3] + np.random.normal(bias,2,1)
        zd2 = np.array([d[0],d[1],d[2],d[3]])
        self.m = zd2
        print('Init mean')
        print(self.m)
        print('Init s2')
        print(self.s2)

    def get_elbo(self):
        t1 = np.log(self.s2) - self.m/self.sigma2
        t1 = t1.sum()
        t2 = -0.5*np.add.outer(self.X**2, self.s2+self.m**2)
        t2 += np.outer(self.X, self.m)
        t2 -= np.log(self.phi)
        t2 *= self.phi
        t2 = t2.sum()
        return t1 + t2

    def fit(self, max_iter=100, tol=1e-10):
        self._init()
        self.elbo_values = [self.get_elbo()]
        self.m_history = [self.m]
        self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
            self._cavi()
            self.m_history.append(self.m)
            self.s2_history.append(self.s2)
            self.elbo_values.append(self.get_elbo())
            if iter_ % 5 == 0:
                print(iter_, self.m_history[iter_])
            if np.abs(self.elbo_values[-2] - self.elbo_values[-1]) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.elbo_values[-1],
                                                                     iter_))
                break

        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.elbo_values[-1]))


    def _cavi(self):
        self._update_phi()
        self._update_mu()

    def _update_phi(self):
        t1 = np.outer(self.X, self.m)
        t2 = -(0.5*self.m**2 + 0.5*self.s2)
        exponent = t1 + t2[np.newaxis, :]
        self.phi = np.exp(exponent)
        self.phi = self.phi / self.phi.sum(1)[:, np.newaxis]

    def _update_mu(self):
        self.m = (self.phi*self.X[:, np.newaxis]).sum(0) * (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.m.size == self.K
        #print(self.m)
        self.s2 = (1/self.sigma2 + self.phi.sum(0))**(-1)
        assert self.s2.size == self.K
        
def tri(zd, H):
    zd = zd.T
    H=H-H[0]
    tmp = zd[0]
    tmp = tmp * tmp
    zd = np.delete(zd,(0), axis=0)
    r1=np.multiply(zd,zd)
    Ha=np.delete(H,(0), axis=0)
    K=np.multiply(Ha,Ha)
    K=K.sum(axis=1)
    b= 0.5*(K-r1+tmp) 
    S_inv = np.linalg.inv(Ha.T*Ha)*Ha.T
    x_hat = S_inv*b
    return x_hat  

#num_components = 4
#mu_arr = np.random.choice(np.arange(-10, 10, 2),
#                          num_components) +\
#         np.random.random(num_components)
a = 1
bias = 0
std = 2
x = np.array([7.54,4.8])
for q in range(0,100):
    x = np.array([15*random.random(),15*random.random()])
    print(x)
    H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
    d = x-H
    d = np.multiply(d,d)
    d = np.sqrt(d.sum(axis=1))
    print('\n')
    
    d[0] = d[0] + np.random.normal(1,2,1)
    d[1] = d[1] + np.random.normal(1,2,1)
    d[2] = d[2] + np.random.normal(1,2,1)
    d[3] = d[3] + np.random.normal(1,2,1)
    zd3 = np.array([d[0],d[1],d[2],d[3]])
    ##### Generating Gaussian Mixture Model #####
    d[0] = d[0] + np.random.normal(bias,1,1)
    d[1] = d[1] + np.random.normal(bias,1,1)
    d[2] = d[2] + np.random.normal(bias,1,1)
    d[3] = d[3] + np.random.normal(bias,1,1)
    zd2 = np.array([d[0],d[1],d[2],d[3]])

    mu_arr = zd3
    SAMPLE = 1000
    X = np.random.normal(loc=mu_arr[0], scale=1, size=SAMPLE)
    for i, mu in enumerate(mu_arr[1:]):
        X = np.append(X, np.random.normal(loc=mu, scale=1, size=SAMPLE))

    ugmm = UGMM(X, 4)
    ugmm.fit()
    #mu_arr.sort()
    ugmm.m.sort()
    
    print(ugmm.m)
    #zd = np.matrix([ugmm.m[1],ugmm.m[0],ugmm.m[3],ugmm.m[2]])
    #########################################################
    # Sorting the order of the results for multilateration #
    l = mu_arr.size
    t = np.ones((1,l))
    print(mu_arr)
    for i in range(0,l):
        k = np.argmin(mu_arr)
        t[0][k] = ugmm.m[i]
        mu_arr[k]=100
    print(t)
    ########################################################

    zd = np.matrix([t[0][0],t[0][1],t[0][2],t[0][3]])
    x_hat = tri(zd,H2)
    x_hat = x_hat.T+H[0]
    print(x_hat)
    print('\n')
    #######################################################################
    ############ Plotting Culmulative Distribution Funcfion ###############
    dev = np.multiply((x_hat-x),(x_hat-x))
    tmp = dev.item(0)+dev.item(1)
    res = np.sqrt(tmp)
    x_hat = tri(zd3,H2)
    x_hat = x_hat.T+H[0]
    dev = np.multiply((x_hat-x),(x_hat-x))
    tmp = dev.item(0)+dev.item(1)
    res2 = np.sqrt(tmp)
    write_ws['A1'] = 'Deviation'
    write_ws.cell(a+1,1,res)
    write_ws.cell(a+1,2,res2)
    write_wb.save('/Users/seungwoo/Dropbox/내 PC (DESKTOP-7MKL25L)/Desktop/deviation.xlsx')#('/Users/swbaek/Desktop/deviation.xlsx')
    a = a+1
    #######################################################################


fig, ax = plt.subplots(figsize=(15, 4))
sns.distplot(X[:SAMPLE], ax=ax, hist=True,rug=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[0], 1, SAMPLE), color='k', hist=False, kde=True)
sns.distplot(X[SAMPLE:SAMPLE*2], ax=ax, hist=True,rug=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[1], 1, SAMPLE), color='k', hist=False, kde=True)
sns.distplot(X[SAMPLE*2:SAMPLE*3], ax=ax, hist=True,rug=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[2], 1, SAMPLE), color='k', hist=False, kde=True)
sns.distplot(X[SAMPLE*3:], ax=ax, hist=True,rug=True, norm_hist=True)
sns.distplot(np.random.normal(ugmm.m[2], 1, SAMPLE), color='k', hist=False, kde=True)

[5.03250892 4.81789495]


Init mean
[11.29411056  4.7942678  14.30619775  7.03016254]
Init s2
[0.41114611 0.22773748 0.36831496 0.82853141]
5 [11.73324825  3.27522731 12.73549187  8.28212533]
10 [11.85380437  3.2771192  12.66974395  8.31392428]
15 [11.89150514  3.27754098 12.64442258  8.32179925]
20 [11.90654451  3.27768299 12.63371218  8.32461146]
25 [11.91312916  3.27774112 12.6289281   8.32578887]
30 [11.91612684  3.27776682 12.62673208  8.32631445]
35 [11.91751557  3.27777857 12.62571099  8.32655577]
40 [11.9181641   3.27778403 12.62523335  8.32666799]
45 [11.9184681   3.27778658 12.62500928  8.32672049]
50 [11.91861084  3.27778777 12.62490403  8.32674512]
55 [11.91867793  3.27778833 12.62485455  8.32675669]
60 [11.91870947  3.2777886  12.62483129  8.32676213]
65 [11.9187243   3.27778872 12.62482036  8.32676469]
70 [11.91873127  3.27778878 12.62481521  8.32676589]
75 [11.91873455  3.27778881 12.62481279  8.32676645]
80 [11.91873609  3.27778882 12.62481166  8.32676672]
85 [11.918736

FileNotFoundError: [Errno 2] No such file or directory: '/Users/seungwoo/Dropbox/내 PC (DESKTOP-7MKL25L)/Desktop/deviation.xlsx'

In [1]:
print(mu_arr.size())

NameError: name 'mu_arr' is not defined

In [ ]:
        bias = 1
        std = 2
        x = np.array([7.54,4.8])
        x = np.array([15*random.random(),15*random.random()])
        #random.random()
        H = np.array([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
        H2 = np.matrix([[7.54,0],[7.54,7.21],[14.14,0],[14.14,7.58]])
        d = x-H
        d = np.multiply(d,d)
        d = np.sqrt(d.sum(axis=1))
        d[0] = d[0] + np.random.normal(bias,2,1)
        d[1] = d[1] + np.random.normal(bias,2,1)
        d[2] = d[2] + np.random.normal(bias,2,1)
        d[3] = d[3] + np.random.normal(bias,2,1)
        zd2 = np.array([d[0],d[1],d[2],d[3]])

In [44]:
dev = np.multiply((x_hat-x),(x_hat-x))
tmp = dev.item(0)+dev.item(1)
res = np.sqrt(tmp)
print(res)

0.2466093299604199


In [54]:
print(x_hat.item(0))

2.296672546702662
